# QIIME2 exploratory data analysis workshop

This workshop will guide you through a basic data processing pipeline using QIIME2 on a microbiome dataset. A large proportion will be exploratory data analysis, followed by some more specific statistical analysis and plotting of data.

By-and-large this workshop follows the steps in the [QIIME2 overview tutorial "moving pictures"](https://docs.qiime2.org/2019.7/tutorials/moving-pictures/). There are many more tutorials available on the [QIIME2 tutorial website](https://docs.qiime2.org/2019.7/tutorials/).

## Some basic QIIME2 concepts

For a detailed overview of QIIME2's concepts, [please consult the excellent documentation](https://docs.qiime2.org/2019.7/concepts/).

QIIME2 only handles most "raw" data (such as FASTQ files) on import and bundles data into `.qza` files - QIIME zipped *artifact*. In addition to the actual data, this includes meta-information, such as type of data and provenance.

*Visualisations* on the other hand are wrapped in `.qzv` files - QIIME zipped *visualisation*, with similar meta-information as *artifacts*. *Visualisation* files are usually the terminal output of a process or analysis.

QIIME2's extensive functionality is built with *plugins*. Each *plugin* might wrap specific programmes in other programming languages (such as C or Fortran) and generally fullfills one aspect of data processing or analysis. You can find a [full list of QIIME2 plugins on the website](https://docs.qiime2.org/2019.7/plugins/available/). Throughout this workshop you will also find links to corresponding tutorials.

The structure of QIIME2 commands is always similar, starting with `qiime` followed by the *plugin* name and several flags for options and file input or outputs. You can access the documentation by adding the `--help` flag, e.g.

In [ ]:
!qiime tools import --help

Note the exclaimation point (`!`) at the beginning of the command. This invokes the command as a `shell` operation rather than the default Python. Alternatively, you can use `%%bash` at the top of the cell to turn the cell into a `shell/bash` script.

You will frequently find `%%time` throughout this workshop, which will give you and idea how long it takes to execute code.

## Viewing *Visualisations* in Jupyter

*Visualisations* can be viewed with a QIIME view plugin, however, as we are working on a supercomputer, a so called 'headless' system without a graphic user interfact. Hence, throughout the analysis, we will do a little work-around using QIIME's `Visualization` Python package (note the American spelling).

We will therefore need to import the package.

In [ ]:
from qiime2 import Visualization

*Visualisation* files can then be loaded through a Python command:

```python
Visualization.load('visualisation-file.qzv')
```

This will load the *Visualisation* in the output for the corresponding cell. Each *Visualisation* will also have an "open in new window" link to display the output in a new window/tab.

## The dataset and metadata

You will analyse an oral microbiome dataset. A total of 12 samples were collected by swabing the mouth between 8:00 and 18:00; one before brushing teeth, then hourly. The subject was a 21 year old male with good dental health on a vegan diet.

All metadata for this experiment has been [colated in a spreadsheet](https://docs.google.com/spreadsheets/d/1kdIW4xIxysbf0zQBzO4Y65B9sfaeTZywZ1CrMwWMd9k/edit?usp=sharing). Metadata files follow specific formatting rules (https://docs.qiime2.org/2019.7/tutorials/metadata/) and data included in the files can be used in some of the analyses by QIIME2.

To validate the metadata format, you can either use a QIIME2 plugin, or, more conveniently, use a Google sheets plugin called `Keemei` (Add-ons > Get Add-ons in the menu). Once validated, export the file as a tab-separated value (`.tsv`) file (under File > Download in the menu) and upload it to your working directory (drag and drop onto the files panel of Jupyter).

## Importing data with manifest

https://docs.qiime2.org/2019.7/tutorials/importing/

You will work a fraction (~5%) of the total dataset for this particular experiment (about a 1/400th of a full HiSeq run) to reduce computation time.

Let's copy the data to your working directory.

In [ ]:
!cp -rv /home/mbaron/qiime2_workshop/pe-reads/ .

Each of the samples is represented by one forward and one reverse read `.fastq` file - paired end reads. This means the data is already demultiplexed. The `manifest.txt` file in the `pe-reads` folder contains the sample ids and the file paths to each of the files.

To import the data as an *artifact*, `tools import` needs to be informed about the type of data (paired end sequences with quality information), the encoding of the quality scores (can be found in `metadata.yml`) and the filepaths through the manifest.

The import process will take a couple of seconds to a minute and create a `.qza` file.

In [ ]:
%%time
!qiime tools import \
    --type 'SampleData[PairedEndSequencesWithQuality]' \
    --input-path pe-reads/manifest.txt \
    --output-path demux \
    --input-format PairedEndFastqManifestPhred33V2

You can take a brief look at the file type and format using `tools peek`.

In [ ]:
%%time
!qiime tools peek demux.qza

### Visualising demultiplexed data

Once the data is imported, we can visualise the quality score information by creating a `.qzv` file. There are lots of different visualisation methods associated with different `.qza` files (see [available plugins](https://docs.qiime2.org/2019.7/plugins/available/)). `demux summarize` will provide you with an overview of sequences for each sample and an interactive chart displaying quality scores vs sequence length.

Do you see a marked drop in sequence quality?

In [ ]:
%%time
!qiime demux summarize \
    --i-data demux.qza \
    --o-visualization demux.qzv

In [ ]:
Visualization.load('demux.qzv')

## Picking Amplicon Sequence Variants (ASV) with Deblur

The [Deblur](http://msystems.asm.org/content/2/2/e00191-16) algorithm uses error profiles to denoise the sequencing data and find true biological variants. We will quality filter our data by first by joining the paired-end reads. Any remaining low-quality read are removed a Q-score based filtering approach (as described in the lecture).

### Joining paired-end reads

https://docs.qiime2.org/2019.7/plugins/available/vsearch/join-pairs/

The expected size of the region amplified by the 515f and 806r primers is close to 250bp. As the data was generated by a HiSeq Rapid-Run with 250bp reads, the overlap between forward and reverse sequences is likely substaintial. `-p-minovlen` describes the minimum overlap between the reads, while `-p-maxdiffs` sets how many differences are tolerated. The default settings of the `vsearch` algorithm are rather lax, so the overlap was set to a minimum of 50bp (which would allow a total length of 450bp); no differences were tolerated.

In [ ]:
%%time
!qiime vsearch join-pairs \
    --i-demultiplexed-seqs demux.qza \
    --output-dir joined \
    --p-minovlen 50 \
    --p-maxdiffs 0

In [ ]:
%%time
!qiime demux summarize \
    --i-data joined/joined_sequences.qza \
    --o-visualization joined/joined_sequences.qzv

In [ ]:
Visualization.load('joined/joined_sequences.qzv')

### Quality filtering by Q-score

https://docs.qiime2.org/2019.7/plugins/available/quality-filter/q-score-joined/

In [ ]:
%%time
!qiime quality-filter q-score-joined \
    --i-demux joined/joined_sequences.qza \
    --output-dir filtered

In [ ]:
%%time
!qiime demux summarize \
    --i-data filtered/filtered_sequences.qza \
    --o-visualization filtered/filtered_sequences.qzv

In [ ]:
Visualization.load('filtered/filtered_sequences.qzv')

In [ ]:
%%time
!qiime metadata tabulate \
  --m-input-file filtered/filter_stats.qza \
  --o-visualization filtered/filter_stats.qzv

In [ ]:
Visualization.load('filtered/filter_stats.qzv')

### Deblur

https://docs.qiime2.org/2019.7/plugins/available/deblur/denoise-16S/

With larger datasets, the actual Deblur step can be very time-consuming. With the current relatively small dataset, it should complete with a few minutes on the supercomputer.

This script can run in parallel on several CPU cores by using the `--p-jobs-to-start` flag. You will come across other plugins, which can also be parallelised, though the flags might be called differently, containg works like `threads` or `cores`. The majority of Cartesius compute nodes have 24 cores available.

The output is a feature table, statistics on the denoising and ASV-picking process as well as a file only containing key representative sequences (ASVs). The representative sequences, as a smaller file, allow faster construction of a phylogenetic tree or assignment of taxonomy.

In [ ]:
%%time
!qiime deblur denoise-16S \
    --i-demultiplexed-seqs filtered/filtered_sequences.qza \
    --p-trim-length 251 \
    --output-dir deblur \
    --p-sample-stats \
    --p-jobs-to-start 24

In [ ]:
%%time
!qiime deblur visualize-stats \
    --i-deblur-stats deblur/stats.qza \
    --o-visualization deblur/stats.qzv

In [ ]:
Visualization.load('deblur/stats.qzv')

## Assigning taxonomy using the [Human Oral Microbiome Database (HOMD)](http://www.homd.org/index.php)

https://docs.qiime2.org/2019.7/tutorials/feature-classifier/

There are many references databases, some of the more popular general references, such as GreenGenes and Silva, can be found through the [QIIME data resources page](https://docs.qiime2.org/2019.7/data-resources/).

Using a more specialised a database has the advantage of more reliable and accurate taxonomy assignment. More specialised databases are usually smaller and allowing faster processing.

### Importing HOMD data

Two files are required to build a taxonomy classifier, the references sequences in `.fasta` format and text-file file liking the reference IDs with taxa. Both these files can be found on [HOMD ftp download pages for their 16S rRNA references](http://www.homd.org/ftp/16S_rRNA_refseq/HOMD_16S_rRNA_RefSeq/V15.2/). Let's make a new directory and download them directly using `wget`.

In [ ]:
!mkdir homd
!wget -P homd "http://www.homd.org/ftp/16S_rRNA_refseq/HOMD_16S_rRNA_RefSeq/V15.2/HOMD_16S_rRNA_RefSeq_V15.2.fasta"
!wget -P homd "http://www.homd.org/ftp/16S_rRNA_refseq/HOMD_16S_rRNA_RefSeq/V15.2/HOMD_16S_rRNA_RefSeq_V15.2.qiime.taxonomy"

For importing, `tools import` does the job. Note that we specifying different `--type` for each of the imports.

In [ ]:
%%time
# importing HOMD data
!qiime tools import \
    --type 'FeatureData[Sequence]' \
    --input-path homd/HOMD_16S_rRNA_RefSeq_V15.2.fasta \
    --output-path homd/HOMD_16S_rRNA_RefSeq_V15.2

In [ ]:
%%time
!qiime tools import \
    --type 'FeatureData[Taxonomy]' \
    --input-format HeaderlessTSVTaxonomyFormat \
    --input-path homd/HOMD_16S_rRNA_RefSeq_V15.2.qiime.taxonomy \
    --output-path homd/HOMD_16S_rRNA_RefSeq_V15.2.qiime.taxonomy

### Trimming references with primer sequences

https://docs.qiime2.org/2019.7/plugins/available/feature-classifier/extract-reads/

Trimming the reference database to the region of interest (in our case between 515f and 806r), also ensure more reliable assignment of taxonomy. To do so, the plugin requires the annealing regions of both primer sequences (note the degenerate bases e.g `M`) and minimum and maximum expected sequence lengths. As the region should be around 250 bp, a minimum length of 100 bp and a maximum length of 400 bp should reduce spurious sequences.

In [ ]:
%%time
## extracting reads according to primers
!qiime feature-classifier extract-reads \
    --i-sequences homd/HOMD_16S_rRNA_RefSeq_V15.2.qza \
    --p-f-primer GTGCCAGCMGCCGCGGTAA \
    --p-r-primer GGACTACHVGGGTWTCTAAT \
    --p-min-length 100 \
    --p-max-length 400 \
    --o-reads homd/HOMD_16S_rRNA_RefSeq_V15.2_trimmed.qza

### Assigning taxonomy

https://docs.qiime2.org/2019.7/plugins/available/feature-classifier/classify-consensus-vsearch/

VSEARCH is used to align our query sequences (remember the representative sequences generated by Deblur?) with the reference and assign taxonomy.

In [ ]:
%%time
!qiime feature-classifier classify-consensus-vsearch \
    --i-query deblur/representative_sequences.qza \
    --i-reference-reads homd/HOMD_16S_rRNA_RefSeq_V15.2_trimmed.qza \
    --i-reference-taxonomy homd/HOMD_16S_rRNA_RefSeq_V15.2.qiime.taxonomy.qza \
    --output-dir taxonomy \
    --p-threads 24

### Taxa bar-plot

https://docs.qiime2.org/2019.7/plugins/available/taxa/barplot/

A convenient way to visualise taxonomy is through stacked bar-charts. Do you see any trends in any taxa throughout the day? We'll get back to the taxonomy data later.

In [ ]:
%%time
!qiime taxa barplot \
    --i-table deblur/table.qza \
    --i-taxonomy taxonomy/classification.qza \
    --m-metadata-file oral_microbiome.tsv \
    --o-visualization taxonomy/bar-plot.qzv

In [ ]:
Visualization.load('taxonomy/bar-plot.qzv')

## Generating a phylogenetic tree

https://docs.qiime2.org/2019.7/plugins/available/phylogeny/align-to-tree-mafft-fasttree/

Several phylogeny-based diversity metrics require a phylogenetic tree. This pipeline method will align all the sequences, denoise the data and construct both a rooted and unrooted phylogenetic tree. Again, using only the representative sequences speeds up this process.

In [ ]:
%%time
!qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences deblur/representative_sequences.qza \
    --p-n-threads 24 \
    --output-dir phylogeny

### Visualise phylogentic tree

QIIME2 doesn't yet have plugins to visualise phylogenetic trees. If you like to see the tree, download your `phylogeny/tree.qza` file (right-click on the file in the file panel and download) and upload it to [Interactive Tree Of Life](https://itol.embl.de/upload.cgi), a tool by maintained by the EMBL.

For full annotation of the leafs, also download `taxonomy/classification.qza` and drag and drop the file onto the tree. The [iTOL help pages list several other QIIME2 artifacts which can be applied to the tree](https://itol.embl.de/help.cgi#qiime).

## Alpha and beta diversity

https://docs.qiime2.org/2019.7/plugins/available/diversity/core-metrics-phylogenetic/

The core metrics pipeline is a handy tool to generate a whole array of alpha and beta diversity metrics. [This excellent post on the QIIME2 forum](https://forum.qiime2.org/t/alpha-and-beta-diversity-explanations-and-commands/2282) gives a brief overview with papers for each available metric. As mentioned in the lecture, there are many metrics, many of which originating form the much older field of echology. Given that we work with sequencing data you might want to focus at first on phylogenetic metrics, such as [Faith's Phylogenetic Distance](https://en.wikipedia.org/wiki/Phylogenetic_diversity) for alpha diversity and UniFrac for beta diversity.

The pipeline required a rooted phylogenetic tree, the feature table, the meta-data file, as well as a sampling-depth parameter. Each sample will be randomly subsampled (rarefied) to the depth of `--p-sampling-depth` to ensure even representation. Any sample smaller than the sampling depth will be excluded. Consequently, this is a tradeoff between depth of the analysis and number of samples retained.

You need to check your visualisation of the Deblur-statistics for sampling depth. Sort the data by the `read-hit-reference`. Is there a large difference between the most and least numerous sample?

In [ ]:
%%time
!qiime diversity core-metrics-phylogenetic \
    --i-phylogeny phylogeny/rooted_tree.qza \
    --i-table deblur/table.qza \
    --p-sampling-depth 18435 \
    --m-metadata-file oral_microbiome.tsv \
    --output-dir alpha_beta \
    --p-n-jobs 24

[Ordination](https://en.wikipedia.org/wiki/Ordination_(statistics)) is a popular tool to explore microbial communities. We can use the [Emperor tool](https://biocore.github.io/emperor/description_index.html) to inspect [principal coordinate analysis (PCoA)](https://moodle.ucl.ac.uk/mod/page/view.php?id=988175) plots. Due to a bug in the QIIME2 implementations data labels can not be set, however when you click on a sample point, its label is revealed in the bottom left corner of the plot.

In [ ]:
Visualization.load('alpha_beta/jaccard_emperor.qzv')

### Alpha diversity correlation

https://docs.qiime2.org/2019.7/plugins/available/diversity/alpha-correlation/

Given that this dataset is a time-series, it makes sense to look for correlation over time. The alpha-correlation method will determine whether any numeric data provided in the metadata file are correlated with alpha diversity. By default it will apply a [Spearman's rank correlation test](https://statistics.laerd.com/spss-tutorials/spearmans-rank-order-correlation-using-spss-statistics.php), generate a correlation coefficient (-1 to 1) and a p-value. Is the correlation significant?

In [ ]:
%%time
!qiime diversity alpha-correlation \
  --i-alpha-diversity alpha_beta/faith_pd_vector.qza \
  --m-metadata-file oral_microbiome.tsv \
  --o-visualization alpha_beta/faith_pd_correlation

In [ ]:
Visualization.load('alpha_beta/faith_pd_correlation.qzv')

### Alpha rarefaction curves

https://docs.qiime2.org/2019.7/plugins/available/diversity/alpha-rarefaction/

Rarefaction curves are generated by randomly sub-sampling data from each sample at increasing depth (up to `-p-max-depth`). Each sub-sampling is repeated several times (default: 10) and alpha diversity metric are calculated. As the sampling depth increases, so should the diversity indices. If the curves plateau or level out, increasing sequencing depth would unlikely increase the number of features deteced.

Hence, alpha rarefaction curves are a tool to determine whether most of the diversity had been captured through sequencing.

In [ ]:
%%time
!qiime diversity alpha-rarefaction \
    --i-table deblur/table.qza \
    --i-phylogeny phylogeny/rooted_tree.qza \
    --p-max-depth 18435 \
    --m-metadata-file oral_microbiome.tsv \
    --p-steps 20 \
    --o-visualization alpha_beta/rarefaction

In [ ]:
Visualization.load('alpha_beta/rarefaction.qzv')

## Further correlation analysis of taxonomic data with Python

The significant decrease in alpha diversity throughout the day suggests that taxa change significantly throughout the day. Usually a drop in diversity is associated with some taxa increasing relative abundance, ie. outcompeting others.

What better way to test this than with a few quick correlation tests in Python through the [Pandas](https://pandas.pydata.org/) packages.

First we'll export the bar-plot data and import with Pandas.

In [ ]:
%%time
!qiime tools export \
    --input-path taxonomy/bar-plot.qzv \
    --output-path taxonomy/bar-plot

In [ ]:
# loading pandas package
import pandas as pd
# jupyter magic to display plots inline
%matplotlib inline

In [ ]:
# loading data into a Pandas dataframe
# 'time' is set as the row-index column, times are parsed by Python
df = pd.read_csv('taxonomy/bar-plot/level-6.csv',index_col='time',parse_dates=True)

In [ ]:
# sorting data by index (inplace to overwrite dataframe)
df.sort_index(inplace=True)
# rows are now sorted by time / sample
df.index

In [ ]:
# provides overview of dataframe
print(df.info())
# shows first couple of rows of dataframe
df.head()

In [ ]:
# data is in absolute counts
# to get relative abundances, the data needs to be normalised
# create a selector for the columns (in this case for the first and last taxonomy column in the dataframe)
start, end = df.columns.get_indexer(['Unassigned;__;__;__;__;__','k__Bacteria;p__Spirochaetes;c__Spirochaetia;o__Spirochaetales;f__Spirochaetaceae;g__Treponema'])
taxa_columns = df.columns[start:end]
df['all'] = df.loc[:,taxa_columns].sum(axis=1)
# divide each taxonomy count by the total value, mulitply by 100
df.loc[:,taxa_columns] = df.loc[:,taxa_columns].div(df['all'],axis=0).mul(100,axis=0)

In [ ]:
df.head()

In [ ]:
# lets create a quick plot to visualise how taxa are changing over time
# the legend is hidden, as it would just cover the whole plot
df.plot(x = 'time-past-brushing-hours', y = taxa_columns, legend=False)

In [ ]:
# It appears that some taxa change quite significantly
# Let's figure out which ones change the most with a few quick operations
# Here we calulate the difference between maximum and mimium for each column and sort them
abund_changes = (df[taxa_columns].max(axis=0)-df[taxa_columns].min(axis=0))
abund_changes.sort_values(ascending=False).head(10)

In [ ]:
# it appears that Streptococcus and Neisseria experience quite some swing throughout the day
# let's see whether their correlation coefficients are also reasonably large
# with the .corrwith function we can correlate several data series (columns) with another
corr_coeff = df[taxa_columns].corrwith(df['time-past-brushing-hours'])
# putting both series into one dataframe for easier intepretation
df2 = pd.concat([abund_changes, corr_coeff],axis=1)
df2.sort_values(by=0,ascending=False).head(10)

In [ ]:
# This looks promising, let's plot some of the data to visualise
df.plot(x='time-past-brushing-hours', y = 'k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Streptococcaceae;g__Streptococcus', kind='line')
df.plot(x='time-past-brushing-hours', y = 'k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Neisseriales;f__Neisseriaceae;g__Neisseria', kind='line')
df.plot(x='time-past-brushing-hours', y = 'k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Burkholderiales;__;__', kind='line')

Simple correlation analysis can be done directly on the dataframe

In [ ]:
# For more a thorough correlation analysis we need to import the stats package from scientific python (scipy)
from scipy import stats

In [ ]:
# we can now carry out individual correlation tests on each of the top 3 taxa
# this will also provide a p-value
R, p = stats.pearsonr(df['k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Streptococcaceae;g__Streptococcus'],df['time-past-brushing-hours'])
print(f'The correlation coefficient is: {R}')
print(f'The p-value is: {p}')
stats.spearmanr(df['k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Streptococcaceae;g__Streptococcus'],df['time-past-brushing-hours'])

In [ ]:
R, p = stats.pearsonr(df['k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Neisseriales;f__Neisseriaceae;g__Neisseria'],df['time-past-brushing-hours'])
print(f'The correlation coefficient is: {R}')
print(f'The p-value is: {p}')
# or a non-parametric correlation test
stats.spearmanr(df['k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Neisseriales;f__Neisseriaceae;g__Neisseria'],df['time-past-brushing-hours'])

In [ ]:
R, p = stats.pearsonr(df['k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Burkholderiales;__;__'],df['time-past-brushing-hours'])
print(f'The correlation coefficient is: {R}')
print(f'The p-value is: {p}')
# or a non-parametric correlation test
stats.spearmanr(df['k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Burkholderiales;__;__'],df['time-past-brushing-hours'])

It appears that there is a strong and significant correlation between *Streptococcus* relative abundance and time after brushing teeth. 

The correlation in *Neisseira* abundance proofed non-significant (p > 0.05) and less strong. Though starting the correlation at a slightly later timepoint would probably provide different results.

Lastly, an unspecified genus from the order *Burkholderiales* also exhibits a strong, signficant correlation

The next step would now be to find biological reasons for these correlations.

**Well done, you are finished with this workshop.** You can now shutdown the notebook, or use the remaining time to explore other [QIIME2 tutorials](https://docs.qiime2.org/2019.7/tutorials/).